In [ ]:
from streaming_capymoa import capymoa_classifier, save_results
from pham_ensemble_capymoa import PhamEnsemble, save_results_pham
from random_projection import create_dataset_RP
import os
import pandas as pd
import pickle

#script_dir = os.path.dirname(os.path.abspath(__file__))
#Set the current working directory to the script's directory
#os.chdir(script_dir)
print(os.getcwd())

In [2]:
def retrieve_data_dir(feature_type, data_dir_imagenet, data_dir_moco, data_dir_moco_ft):
    if feature_type=='imagenet':
        data_dir = data_dir_imagenet
    elif feature_type=='moco':
        data_dir = data_dir_moco
    elif feature_type=='moco_ft':
        data_dir = data_dir_moco_ft
    else: raise ValueError("The provided feature type is not supported. Please prove imagenet, moco or moco_ft")
    return data_dir

def results_to_df(classifier_capymoa, results_df, time_rp, no_RP = False):
    if results_df is None:
        column_names = ['classifier', 'features', 'k', 'type_RP', 'no_RP','prequential', 'in_domain', 'next_domain', 'ft', 'bt',
                         'time_classifier', 'time_ft', 'time_bt', 'seed', 'time_rp']
        results_df = pd.DataFrame(columns=column_names)
    new_row = {}
    new_row['classifier'] = classifier_capymoa.classifier
    new_row['features'] = classifier_capymoa.rp_settings['features']
    new_row['k'] = classifier_capymoa.rp_settings['n_reduced']
    new_row['type_RP'] = classifier_capymoa.rp_settings['type_RP']
    new_row['no_RP'] = no_RP    
    new_row['prequential'] = classifier_capymoa.result_cumulative_pd['accuracy'].iloc[-1]
    new_row['in_domain'] = classifier_capymoa.in_domain_acc
    new_row['next_domain'] = classifier_capymoa.next_domain_acc
    new_row['ft'] = classifier_capymoa.ft_acc
    new_row['bt'] = classifier_capymoa.bt_acc
    new_row['time_classifier'] = classifier_capymoa.time_classifier/60
    new_row['time_ft'] = classifier_capymoa.time_ft/60
    new_row['time_bt'] = classifier_capymoa.time_bt/60
    new_row['seed'] = classifier_capymoa.seed
    new_row['time_rp'] = time_rp
    new_row_df = pd.DataFrame([new_row])
    results_df = pd.concat([results_df, new_row_df], ignore_index=True)
    return(results_df)

In [3]:

def multiple_runs_k_values(superclass_dict, rp_types, feature_types, seeds, classifiers, model_names, k_values, data_dirs, folder_save,
                            save_df, first_exp=0, nexp=11):
    results_df = None
    idx=0
    for feature_type in feature_types:
        data_dir = retrieve_data_dir(feature_type=feature_type, data_dir_imagenet=data_dirs['imagenet'], data_dir_moco=data_dirs['moco'],
                                      data_dir_moco_ft=data_dirs['moco_ft'])
        for classifier in classifiers:
            for rp_type in rp_types:
                for k in k_values:
                    rp_settings = {
                        'n_reduced': k,
                        'type_RP': rp_type,
                        'features': feature_type
                    }

                    for seed in seeds:
                        # data directory in which projected data are saved
                        save_dir = folder_save +'/k_values/rp_dataset/rp_' + feature_type + '_' + rp_type + '_k=' + str(k) + '_seed_' + str(seed)
                        #print(save_dir)
                        if not os.path.exists(save_dir):
                            os.makedirs(save_dir)
                        # project data via random projection
                        time_RP = create_dataset_RP(data_dir=data_dir, save_dir = save_dir, split_data=False, n_original=2048, 
                                                    n_reduced=rp_settings['n_reduced'], type_RP=rp_settings['type_RP'], seed=seed, nexp=nexp)        
                        
                        dir_save_models = folder_save + '/models/k_values_' +classifier + '_'+ feature_type + '_' + rp_type + '_k=' + str(k) + '_seed_' + str(seed)
                        if not os.path.exists(dir_save_models):
                            os.makedirs(dir_save_models)

                        
                        # initialize the classifier
                        classifier_to_run=capymoa_classifier(classifier=classifier, rp_settings=rp_settings, nexp=nexp, window_size=None, first_exp=first_exp,
                                                            superclass_dict=superclass_dict, model_name = model_names[classifier], seed=seed, data_dir=save_dir+'/all',
                                                            dir_save_models = dir_save_models)
                        # run the model
                        classifier_to_run.run_model(show_accuracy=False, show_confusion=False, compute_ft=True, show_ft=False, compute_bt=True)

                        # save the relevant results
                        filename_results_save = classifier + '_'+ feature_type + '_' + rp_type + '_k=' + str(k) + '_seed_' + str(seed)
                        save_results(classifier_capymoa=classifier_to_run, folder_save=folder_save, filename_save=filename_results_save)
                        results_df = results_to_df(classifier_capymoa=classifier_to_run, results_df=results_df, time_rp=time_RP, no_RP=False)
                        results_df.to_csv(folder_save +'/'+ save_df+ str(idx) + '.csv', index=False)
                        idx+=1

    results_df.to_csv(folder_save +'/'+ save_df+ '.csv', index=False)



In [8]:
def multiple_runs_Pham(superclass_dict, rp_types, feature_types, seeds, classifiers, k_values, n_models, data_dirs, folder_save,
                            save_df, first_exp=0, nexp=11):
    results_df = None

    for feature_type in feature_types:
        data_dir = retrieve_data_dir(feature_type=feature_type, data_dir_imagenet=data_dirs['imagenet'], data_dir_moco=data_dirs['moco'],
                                      data_dir_moco_ft=data_dirs['moco_ft'])
        for classifier in classifiers:
            for rp_type in rp_types:
                for k in k_values:
                    rp_settings = {
                        'n_reduced': k,
                        'type_RP': rp_type,
                        'features': feature_type,
                        'n_original':2048
                    }

                    for seed in seeds:        
                        dir_save_models = folder_save + '/models/k_values_' +classifier + '_'+ feature_type + '_' + rp_type + '_k=' + str(k) + '_seed_' + str(seed)
                        if not os.path.exists(dir_save_models):
                            os.makedirs(dir_save_models)

                        
                        # initialize the classifier
                        classifier_to_run = PhamEnsemble(classifier=classifier, n_models = n_models, class_dict = superclass_dict, rp_settings=rp_settings,
                                                         save_dir_models=dir_save_models, data_dir=data_dir, nexp=nexp, first_exp=first_exp, seed = seed)

                        # run the model
                        classifier_to_run.run_model(show_accuracy=False, show_confusion=False, compute_ft=True, compute_bt=True)

                        # save the relevant results
                        filename_results_save = 'Pham_' + classifier + '_'+ feature_type + '_' + rp_type + '_k=' + str(k) + '_seed_' + str(seed)
                        save_results_pham(classifier_capymoa=classifier_to_run, folder_save=folder_save, filename_save=filename_results_save)
                        results_df = results_to_df(classifier_capymoa=classifier_to_run, results_df=results_df, time_rp=0, no_RP=False)
                        results_df.to_csv(folder_save +'/'+ save_df+ '.csv', index=False)


In [ ]:
data_dirs = {
    'imagenet': 'feature_extraction/features_imagenet_seed2024/all',
    'moco': 'feature_extraction/features_moco_seed2024/all',
    'moco_ft': 'feature_extraction/features_moco_ft_seed2024/all'
}

superclass_dict = {
    0: 'airplane',
    1: 'amusement_park',
    2: 'anime',
    3: 'aquarium',
    4: 'backpack',
    5: 'baseball',
    6: 'basketball',
    7: 'bathroom',
    8: 'beer',
    9: 'bicycle',
    10: 'billard',
    11: 'billboard',
    12: 'blackboard',
    13: 'boat',
    14: 'bookstore',
    15: 'bowling_ball',
    16: 'bridge',
    17: 'bus',
    18: 'camera',
    19: 'canned_food',
    20: 'casino',
    21: 'castle',
    22: 'chef',
    23: 'chocolate',
    24: 'church',
    25: 'coins',
    26: 'coser',
    27: 'diving',
    28: 'farm',
    29: 'ferry',
    30: 'field_hockey',
    31: 'firefighter',
    32: 'food_truck',
    33: 'football',
    34: 'fountain',
    35: 'gallery',
    36: 'garage',
    37: 'glasses',
    38: 'gloves',
    39: 'golf',
    40: 'graffiti',
    41: 'guitar',
    42: 'gym',
    43: 'hair_salon',
    44: 'hat',
    45: 'helicopter',
    46: 'highway',
    47: 'horse_riding',
    48: 'ice_cream',
    49: 'ice_hockey',
    50: 'ice_skating',
    51: 'lab',
    52: 'lamppost',
    53: 'laptop',
    54: 'laundry',
    55: 'lego',
    56: 'microphone',
    57: 'motorcycle',
    58: 'mug',
    59: 'necklaces',
    60: 'newspaper',
    61: 'observatory',
    62: 'opera_house',
    63: 'pet_store',
    64: 'piano',
    65: 'plush_toys',
    66: 'policeman',
    67: 'power_plant',
    68: 'racing_car',
    69: 'ring',
    70: 'road_sign',
    71: 'robot',
    72: 'roller_skate',
    73: 'scarf',
    74: 'shopping_mall',
    75: 'skateboarding',
    76: 'skiing',
    77: 'skyscraper',
    78: 'soccer',
    79: 'soldier',
    80: 'stadium',
    81: 'statue',
    82: 'subway',
    83: 'supermarket',
    84: 'surfing',
    85: 'swimming',
    86: 'table_tennis',
    87: 'temple',
    88: 'tennis',
    89: 'tie',
    90: 'tractor',
    91: 'train',
    92: 'umbrella',
    93: 'vase',
    94: 'vending_machine',
    95: 'video_game',
    96: 'violin',
    97: 'volleyball',
    98: 'watch',
    99: 'zoo'
}
superclass_labels = superclass_dict.keys()

model_names = {
    'nb': 'Naive Bayes',
    'ht': 'Hoeffding tree',
    'hat': 'Hoeffding adaptive tree',
    'ht2': 'Hoeffding tree',
    'hat2': 'Hoeffding adaptive tree',
    'knn': 'KNN',
    'knn2': 'Online KNN',
    'sgd2': 'Softmax regression - SGD',
    'ob_ht': 'Online bagging',
    'lb_ht': 'Leveraging bagging',
    'oza': 'OzaBoost',
    'ozaboost': 'OzaBoost'
}

first_exp=0
nexp=11
folder_save = 'k_values'


save_df = 'knn2_very_sparse_imagenet_k=100_ft_bt_seed_2024'
rp_types = ['very sparse']
feature_types = ['imagenet']
seeds = [2024]
classifiers = ['knn2']
k_values = [100]


In [ ]:
multiple_runs_k_values(superclass_dict=superclass_dict, rp_types=rp_types, feature_types=feature_types, seeds=seeds, classifiers=classifiers, 
                       model_names=model_names, k_values=k_values, data_dirs=data_dirs, folder_save=folder_save, first_exp=first_exp, nexp=nexp, 
                       save_df=save_df)

In [ ]:
rp_types = ['sparse']
feature_types = ['moco']
seeds = [2024]
classifiers = ['ht']
k_values = [100]
n_models = 10
folder_save = 'Pham'
save_df = 'Pham_moco_k=100'

multiple_runs_Pham(superclass_dict=superclass_dict, rp_types=rp_types, feature_types=feature_types, seeds=seeds, 
                   classifiers=classifiers, k_values=k_values, n_models=n_models, data_dirs=data_dirs, folder_save=folder_save,
                            save_df=save_df, first_exp=0, nexp=11)